In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src import *
from glob import glob
import librosa
from tabulate import tabulate


In [3]:
# annotated = np.loadtxt('vehicle_count_manual_annotation_with_sound.csv', delimiter=',', dtype=str, usecols=[0,1,2])[1:]

In [4]:
# mask = (annotated[:, 1] != '') & (annotated[:, 2] != '')
# annotated = annotated[mask]

In [5]:
def get_annotations():
    annotated = np.loadtxt('vehicle_count_manual_annotation_with_sound.csv', delimiter=',', dtype=str, usecols=[0,1,2])[1:]
    rows = []
    for name, n_incoming, n_outgoing in annotated:
#         print(name)

        till_min, till_sec = name.split('_')[-1][:-4].split('.')
        from_min, from_sec = name.split('_')[-2].split('.')
        from_time = int(from_min) * 60 + int(from_sec)
        till_time = int(till_min) * 60 + int(till_sec)

        row = [name[:-16], from_time, till_time, int(n_incoming), int(n_outgoing)]
        rows.append(row)
    return rows

annotations = get_annotations()

In [6]:
# files_manual_counts = {}

# for file in files:
#     files_manual_counts[file] = 0
#     for row in annotations:
#         name = row[0]
#         from_time = row[1]
#         if file in name and from_time >= 0:
#             counts = names_counts[name]
#             files_manual_counts[file] += counts
# files_manual_counts

In [7]:
# file_names, manual_counts = annotated[:, 0], annotated[:, 1].astype(int) + annotated[:, 2].astype(int)

# names_counts = {k: v for k, v in zip(file_names, manual_counts)}

In [8]:
files = [
    '20190819-Kutna Hora-L1-out-MVI_0007',
    '20190819-Kutna Hora-L3-in-MVI_0005',
    '20190819-Kutna Hora-L3-out-MVI_0008',
    '20190819-Kutna Hora-L4-in-MVI_0013',
    '20190819-Kutna Hora-L6-out-MVI_0017',
    '20190819-Kutna Hora-L7-out-MVI_0032',
    '20190819-Kutna Hora-L8-in-MVI_0045',
    '20190819-Kutna Hora-L9-in-MVI_0043',
    '20190819-Kutna Hora-L10-out-SDV_1888',
    '20190819-Kutna Hora-L13-in-MVI_0006',
    '20190819-Kutna Hora-L13-out-MVI_0018',
    '20190819-Kutna Hora-L14-out-MVI_0005',
    '20190819-Kutna Hora-L15-out-MVI_0012',
    '20190819-Kutna Hora-L18-in-MVI_0030',
    '20190819-Ricany-L2-in-MVI_0006',
    '20190819-Ricany-L2-out-MVI_0005',
    '20190819-Ricany-L3-in-MVI_0006',
    '20190819-Ricany-L6-in-MVI_0008',
    '20190819-Ricany-L6-out-MVI_0011',
    '20190819-Ricany-L7-in-MVI_0008',
    '20190819-Ricany-L7-out-MVI_0013',
    '20190819-Ricany-L8-in-MVI_0009',
    '20190819-Ricany-L8-out-MVI_0013',
    '20190819-Ricany-L9-in-MVI_0008',
    '20190819-Ricany-L9-out-MVI_0011'
]

In [10]:
# files_manual_counts = {}

# for file in files:
#     files_manual_counts[file] = 0
#     for name in file_names:
#         start_time = name.split('_')[-2]
#         if file in name and float(start_time) >= 0:
#             counts = names_counts[name]
#             files_manual_counts[file] += counts
# files_manual_counts

In [11]:
datapool = DataPool(files, 6, 0.75)

Video loading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [01:02<00:00,  2.49s/it]


In [13]:
files_manual_counts = {}
validation_part_only = False

files_times = {}

for video in datapool:
    file = video.file 
    print(video.file)
    val_time_from = video.get_from_till_time(False)[0]
    intervals = load_intervals(f'data/intervals/{video.file}.MP4.txt')

    if validation_part_only:
        i = np.argmin(np.abs(intervals[:, 0] - val_time_from))
        from_time = intervals[i, 0]
    else:
        from_time = 0
    till_time = intervals[-1, 1]
    
    files_manual_counts[file] = 0
    for row in annotations:
        name = row[0]
        start_time = row[1]
        end_time = row[2]
        if file in name:
            if start_time >= from_time - 3:
                counts = row[3] + row[4]
                files_manual_counts[file] += counts
                
                if file not in files_times:
                    files_times[file] = [start_time, end_time]
                else:
                    files_times[file] = [
                        min(files_times[file][0], start_time), 
                        max(files_times[file][1], end_time)
                    ]
                    
    
    print(from_time, till_time)
    print()

files_manual_counts 

20190819-Kutna Hora-L1-out-MVI_0007
0 1935.02

20190819-Kutna Hora-L3-in-MVI_0005
0 2006.58

20190819-Kutna Hora-L3-out-MVI_0008
0 2045.86

20190819-Kutna Hora-L4-in-MVI_0013
0 2049.3

20190819-Kutna Hora-L6-out-MVI_0017
0 1852.34

20190819-Kutna Hora-L7-out-MVI_0032
0 1904.4

20190819-Kutna Hora-L8-in-MVI_0045
0 1882.84

20190819-Kutna Hora-L9-in-MVI_0043
0 2031.72

20190819-Kutna Hora-L10-out-SDV_1888
0 3743.46

20190819-Kutna Hora-L13-in-MVI_0006
0 1789.14

20190819-Kutna Hora-L13-out-MVI_0018
0 1680.52

20190819-Kutna Hora-L14-out-MVI_0005
0 2021.64

20190819-Kutna Hora-L15-out-MVI_0012
0 1873.3

20190819-Kutna Hora-L18-in-MVI_0030
0 1878.1

20190819-Ricany-L2-in-MVI_0006
0 2001.1

20190819-Ricany-L2-out-MVI_0005
0 1813.1

20190819-Ricany-L3-in-MVI_0006
0 941.58

20190819-Ricany-L6-in-MVI_0008
0 1975.92

20190819-Ricany-L6-out-MVI_0011
0 1815.74

20190819-Ricany-L7-in-MVI_0008
0 1422.26

20190819-Ricany-L7-out-MVI_0013
0 2030.86

20190819-Ricany-L8-in-MVI_0009
0 2016.94

20190819-R

{'20190819-Kutna Hora-L1-out-MVI_0007': 259,
 '20190819-Kutna Hora-L3-in-MVI_0005': 287,
 '20190819-Kutna Hora-L3-out-MVI_0008': 259,
 '20190819-Kutna Hora-L4-in-MVI_0013': 179,
 '20190819-Kutna Hora-L6-out-MVI_0017': 16,
 '20190819-Kutna Hora-L7-out-MVI_0032': 47,
 '20190819-Kutna Hora-L8-in-MVI_0045': 227,
 '20190819-Kutna Hora-L9-in-MVI_0043': 121,
 '20190819-Kutna Hora-L10-out-SDV_1888': 386,
 '20190819-Kutna Hora-L13-in-MVI_0006': 9,
 '20190819-Kutna Hora-L13-out-MVI_0018': 12,
 '20190819-Kutna Hora-L14-out-MVI_0005': 238,
 '20190819-Kutna Hora-L15-out-MVI_0012': 9,
 '20190819-Kutna Hora-L18-in-MVI_0030': 700,
 '20190819-Ricany-L2-in-MVI_0006': 497,
 '20190819-Ricany-L2-out-MVI_0005': 561,
 '20190819-Ricany-L3-in-MVI_0006': 269,
 '20190819-Ricany-L6-in-MVI_0008': 824,
 '20190819-Ricany-L6-out-MVI_0011': 647,
 '20190819-Ricany-L7-in-MVI_0008': 316,
 '20190819-Ricany-L7-out-MVI_0013': 311,
 '20190819-Ricany-L8-in-MVI_0009': 348,
 '20190819-Ricany-L8-out-MVI_0013': 340,
 '20190819-Ri

In [18]:
uuid = 1640285058
model_name='rvce'

validation_part_only = True

model, config = load_model_locally(uuid, model_name=model_name)
transform = create_transformation(config)

files_estimated_counts = {}

for video in datapool:
    file = video.file
    print(video.file)
    intervals = load_intervals(f'data/intervals/{video.file}.MP4.txt')
    from_time = 0
    
    if validation_part_only:
        from_time = files_times[file]
#         val_time_from = video.get_from_till_time(False)[0]
#         i = np.argmin(np.abs(intervals[:, 0] - val_time_from))
#         from_time = intervals[i, 0]
        
    till_time = intervals[-1, 1]
    
    print(from_time, till_time)

    predictions = validate(video.signal, model, transform, config, from_time=from_time, till_time=till_time)
    files_estimated_counts[video.file] = predictions.sum()
    

20190819-Kutna Hora-L1-out-MVI_0007
1567 1935.02
20190819-Kutna Hora-L3-in-MVI_0005
1520 2006.58
20190819-Kutna Hora-L3-out-MVI_0008
1560 2045.86
20190819-Kutna Hora-L4-in-MVI_0013
1539 2049.3
20190819-Kutna Hora-L6-out-MVI_0017
1480 1852.34
20190819-Kutna Hora-L7-out-MVI_0032
1563 1904.4
20190819-Kutna Hora-L8-in-MVI_0045
1413 1882.84
20190819-Kutna Hora-L9-in-MVI_0043
1577 2031.72
20190819-Kutna Hora-L10-out-SDV_1888
2819 3743.46
20190819-Kutna Hora-L13-in-MVI_0006
1620 1789.14
20190819-Kutna Hora-L13-out-MVI_0018
1355 1680.52
20190819-Kutna Hora-L14-out-MVI_0005
1530 2021.64
20190819-Kutna Hora-L15-out-MVI_0012
1553 1873.3
20190819-Kutna Hora-L18-in-MVI_0030
1360 1878.1
20190819-Ricany-L2-in-MVI_0006
1530 2001.1
20190819-Ricany-L2-out-MVI_0005
89 1813.1
20190819-Ricany-L3-in-MVI_0006
288 941.58
20190819-Ricany-L6-in-MVI_0008
1594 1975.92
20190819-Ricany-L6-out-MVI_0011
1567 1815.74
20190819-Ricany-L7-in-MVI_0008
968 1422.26
20190819-Ricany-L7-out-MVI_0013
1590 2030.86
20190819-Rican

In [103]:
rvce_videos, rvce_audio = [], []

errors_video = []
errors_audio = []

do_not_consider = [
#     '20190819-Ricany-L3-in-MVI_0006',
#     '20190819-Ricany-L6-in-MVI_0008',
#     '20190819-Kutna Hora-L18-in-MVI_0030',
#     '20190819-Kutna Hora-L14-out-MVI_0005'
]

table = ['file name, manual events, video error, video rvce, audio error, audio rvce']

all_manual_events = []
all_automatic_counts = []

# print(f'name {" " * 35} | manual | video | audio')

header = ['file', 'manual', 'video', 'error', 'rvce']
table = []

for file, n_manual_counts in files_manual_counts.items():
    if k in do_not_consider:
        continue
            
    events_times = np.loadtxt(f'data/labels/{file}.MP4.txt')
    events_from = files_times[file][0]
    events_till = files_times[file][1]
    n_automatic_counts = np.sum([(events_times >= events_from) & (events_times <= events_till)])
    all_automatic_counts.append(n_automatic_counts)
                
    error_video = n_automatic_counts - n_manual_counts
    
    all_manual_events.append(n_manual_counts)
    
    errors_video.append(error_video)
    
    rvce_video = abs(n_manual_counts - n_automatic_counts) / n_manual_counts
    rvce_videos.append(rvce_video)
    
    row = [file, n_manual_counts, n_automatic_counts, error_video, f'{rvce_video:.2f}']
           
    table.append(row)

row = [
    'summary', 
    f'{np.mean(all_manual_events):.2f} ± {np.std(all_manual_events):.2f}',
    f'{np.mean(all_automatic_counts):.2f} ± {np.std(all_automatic_counts):.2f}',
    f'{np.mean(errors_video):.2f} ± {np.std(errors_video):.2f}',
    f'{np.mean(rvce_videos):.2f} ± {np.std(rvce_videos):.2f}'
]
table.append(row)
table = np.array(table)

from tabulate import tabulate
table = tabulate(table, headers=header)
print(table)

file                                  manual           video            error           rvce
------------------------------------  ---------------  ---------------  --------------  -----------
20190819-Kutna Hora-L1-out-MVI_0007   259              263              4               0.02
20190819-Kutna Hora-L3-in-MVI_0005    287              278              -9              0.03
20190819-Kutna Hora-L3-out-MVI_0008   259              246              -13             0.05
20190819-Kutna Hora-L4-in-MVI_0013    179              165              -14             0.08
20190819-Kutna Hora-L6-out-MVI_0017   16               16               0               0.00
20190819-Kutna Hora-L7-out-MVI_0032   47               44               -3              0.06
20190819-Kutna Hora-L8-in-MVI_0045    227              202              -25             0.11
20190819-Kutna Hora-L9-in-MVI_0043    121              116              -5              0.04
20190819-Kutna Hora-L10-out-SDV_1888  386              264     

In [181]:
for k, v in datapool.items():
    print(k, v.signal)
    
datapool[k].signal

'0.29'

In [187]:
validate

<function src.utils.validate(signal, model, transform, config, tqdm=<function <lambda> at 0x7f853141c040>, return_probs=False, from_time=None, till_time=None, classification=True)>

In [192]:
rvces = []
errors = []
n_trues = []
n_preds = []

table = []
for i in range(5):
    uuid = f'cross_validation_5_augmentation/{i}'
    model, config = load_model_locally(uuid, 'mae')
    transform = create_transformation(config)
    datapool = DataPool(config.testing_files, config.window_length, config.split_ratio)
#     outputs = validate_datapool(datapool, model, config, None)

    for file, video in datapool.items():        
        from_time = files_times[file][0]
        till_time = files_times[file][1]
        predictions = validate(video.signal, model, transform, config, from_time=from_time, till_time=till_time)

        n_pred = predictions.sum()
        n_preds.append(n_pred)

        n_true = files_manual_counts[file]
        n_trues.append(n_true)

        error = n_pred - n_true
        errors.append(error)

        rvce = abs(n_true - n_pred) / n_true
        rvces.append(rvce)

        row = [file, n_true, n_pred, error, f'{rvce:.2f}']
        table.append(row)
    
header = ['file', 'manual', 'predicted', 'error', 'rvce']
row = [
    'summary', 
    f'{np.mean(n_trues):.2f} ± {np.std(n_trues):.2f}',
    f'{np.mean(n_preds):.2f} ± {np.std(n_preds):.2f}',
    f'{np.mean(errors):.2f} ± {np.std(errors):.2f}',
    f'{np.mean(rvces):.2f} ± {np.std(rvces):.2f}'
]

table.append(row)
table = tabulate(table, headers=header)
print(table)

Video loading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:11<00:00,  2.39s/it]


file                                  manual           predicted        error           rvce
------------------------------------  ---------------  ---------------  --------------  -----------
20190819-Kutna Hora-L1-out-MVI_0007   259              262              3               0.01
20190819-Kutna Hora-L10-out-SDV_1888  386              203              -183            0.47
20190819-Kutna Hora-L14-out-MVI_0005  238              238              0               0.00
20190819-Ricany-L3-in-MVI_0006        269              135              -134            0.50
20190819-Ricany-L9-in-MVI_0008        61               62               1               0.02
20190819-Kutna Hora-L3-in-MVI_0005    287              309              22              0.08
20190819-Kutna Hora-L7-out-MVI_0032   47               34               -13             0.28
20190819-Kutna Hora-L13-in-MVI_0006   9                14               5               0.56
20190819-Kutna Hora-L18-in-MVI_0030   700              473     

In [134]:
table, fancy_table = create_fancy_table(outputs)

In [136]:
with open(f'table.txt', 'w') as file:
    file.write(fancy_table)

In [23]:
mae_video, mae_audio = 0, 0
rvce_video, rvce_audio = [], []

errors_video = []
errors_audio = []

do_not_consider = [
#     '20190819-Ricany-L3-in-MVI_0006',
#     '20190819-Ricany-L6-in-MVI_0008',
#     '20190819-Kutna Hora-L18-in-MVI_0030',
#     '20190819-Kutna Hora-L14-out-MVI_0005'
]

table = ['file name, manual events, video error, video rvce, audio error, audio rvce']

all_manual_events = []

# print(f'name {" " * 35} | manual | video | audio')

header = ['file', 'manual', 'video']

for k, n_manual_counts in files_manual_counts.items():
    if k in do_not_consider:
        continue
        
#     n_events_in_each_interval = np.genfromtxt(f'data/for_manual_annotation/{k}.csv', delimiter=',', skip_header=1)[:, 1]
#     n_automatic_counts = int(n_events_in_each_interval.sum())
    
    events_times = np.loadtxt(f'data/labels/{k}.MP4.txt')
    n_automatic_counts = np.sum([(events_times >= files_times[k][0]) & (events_times <= files_times[k][1])])
    
    # append white spaces
    name = ' ' * 40
    name = list(name)
    for i, ki in enumerate(k):
        name[i] = ki
    name = ''.join(name)
    
    n_estimated_counts = files_estimated_counts[k]
    
#     print(f'{name} | {n_manual_counts:>6.0f} | {n_automatic_counts:>5.0f} | {n_estimated_counts}')
    
    error_video = n_automatic_counts - n_manual_counts
    error_audio = n_estimated_counts - n_manual_counts
    
    all_manual_events.append(n_manual_counts)
    
    errors_video.append(error_video)
    errors_audio.append(error_audio)
    
    mae_video += abs(n_manual_counts - n_automatic_counts)
    rvce_v = abs(n_manual_counts - n_automatic_counts) / n_manual_counts * 100
    rvce_video.append(rvce_v)
    
    mae_audio += abs(n_manual_counts - n_estimated_counts)
    rvce_a = abs(n_manual_counts - n_estimated_counts) / n_manual_counts * 100
    rvce_audio.append(rvce_a)
    
    row = f'{k},{n_manual_counts},{error_video},{rvce_v:.1f},{error_audio},{rvce_a:.1f}'
    table.append(row)
    
table = np.array(table)
print(table)
np.savetxt('table.csv', table, fmt='%s', delimiter=',')

mae_video /= len(files_manual_counts)
# rvce_video = np.mean(rvce_video)

mae_audio /= len(files_manual_counts)
# rvce_audio = np.mean(rvce_audio)

print()
print(f'Events {np.mean(all_manual_events):.2f} ± {np.std(all_manual_events):.2f}')
print()
print(f'Error video {np.mean(errors_video):.2f} ± {np.std(errors_video):.2f}')
print(f'RVCE video  {np.mean(rvce_video):.2f} ± {np.std(rvce_video):.2f}%')
print()
# print(f'MAE audio  {mae_audio:.4f}')
print(f'Error audio {np.mean(errors_audio):.2f} ± {np.std(errors_audio):.2f}')
print(f'RVCE audio {np.mean(rvce_audio):.2f} ± {np.std(rvce_audio):.2f}%')

NameError: name 'files_estimated_counts' is not defined

In [15]:
from cross_validation import cross_validation_error
from train_classification import validate_and_save

In [12]:
uuids = []
for i in range(5):
    uuid = f'cross_validation_5/{i}'
    model, config = load_model_locally(uuid, 'rvce')
    
    datapool = DataPool(config.training_files, config.window_length, config.split_ratio)
    validate_and_save(uuid, datapool, 'trn', True)
    validate_and_save(uuid, datapool, 'val', False)
    
    datapool = DataPool(config.testing_files, config.window_length, config.split_ratio)
    validate_and_save(uuid, datapool, 'tst', None)
#     uuids.append(uuid)
# cross_validation_error(uuids)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:06<00:00,  1.29s/it]


In [20]:
uuids = []
for i in range(5):
    uuid = f'cross_validation_5/{i}'
    uuids.append(uuid)
uuids

['cross_validation_5/0',
 'cross_validation_5/1',
 'cross_validation_5/2',
 'cross_validation_5/3',
 'cross_validation_5/4']

In [21]:
cross_validation_error(uuids)

[array(['0.026', '6', '232', '0.465', '[0, 2069]',
       '20190819-Kutna Hora-L14-out-MVI_0005'], dtype='<U36'), array(['0.030', '8', '265', '0.328', '[0, 3752]',
       '20190819-Kutna Hora-L10-out-SDV_1888'], dtype='<U36'), array(['0.101', '-30', '296', '0.448', '[0, 2067]',
       '20190819-Kutna Hora-L1-out-MVI_0007'], dtype='<U36'), array(['0.127', '-8', '63', '0.064', '[0, 2068]',
       '20190819-Ricany-L9-in-MVI_0008'], dtype='<U36'), array(['0.236', '-108', '457', '0.698', '[0, 2068]',
       '20190819-Ricany-L3-in-MVI_0006'], dtype='<U36'), array(['0.042', '14', '336', '0.434', '[0, 2062]',
       '20190819-Ricany-L8-out-MVI_0013'], dtype='<U35'), array(['0.211', '60', '285', '0.379', '[0, 2059]',
       '20190819-Kutna Hora-L3-in-MVI_0005'], dtype='<U35'), array(['0.234', '-11', '47', '0.050', '[0, 2062]',
       '20190819-Kutna Hora-L7-out-MVI_0032'], dtype='<U35'), array(['0.353', '-260', '737', '1.221', '[0, 2065]',
       '20190819-Kutna Hora-L18-in-MVI_0030'], dtype='<

In [210]:
cross_validation_error

<function cross_validation.cross_validation_error(uuids, model_name='rvce', prefix='tst')>

In [22]:
from glob import glob

In [25]:
glob('data/video/*.MP4')

['data/video/20190819-Kategorizace-Vinohradska-smer Ven-MVI_0010.MP4',
 'data/video/20190819-Ricany-L5-in-MVI_0007.MP4',
 'data/video/20190819-Kutna Hora-L9-in-MVI_0043.MP4',
 'data/video/17_RX100.MP4',
 'data/video/20190819-Ricany-L2-in-MVI_0006.MP4',
 'data/video/20190819-Kategorizace-Jizni spojka-cam 3-MVI_0009.MP4',
 'data/video/13_RX100.MP4',
 'data/video/20190819-Kutna Hora-L1-out-MVI_0007.MP4',
 'data/video/20190819-Ricany-L8-out-MVI_0013.MP4',
 'data/video/20190819-Kutna Hora-L14-out-MVI_0005.MP4',
 'data/video/10_iphone.MP4',
 'data/video/20190819-Ricany-L4-in-MVI_0007.MP4',
 'data/video/01_Vypadova_A6000.MP4',
 'data/video/20190819-Kutna Hora-L5-in-MVI_0003.MP4',
 'data/video/11_RX100.MP4',
 'data/video/20190819-Kategorizace-Vinohradska-smer Centrum-MVI_0013.MP4',
 'data/video/20190819-Kutna Hora-L6-out-MVI_0017.MP4',
 'data/video/20190819-Kutna Hora-L13-out-MVI_0018.MP4',
 'data/video/20190819-Kutna Hora-L14-in-SDV_0012.MP4',
 'data/video/20190819-Kutna Hora-L10-out-SDV_1888

In [32]:
import shutil

In [63]:
files = sorted(glob('../data/MultiDo/Eyedea/Prujezdy1/*.csv'))
# files = sorted(glob('data/video/*.MP4'))
files

['../data/MultiDo/Eyedea/Prujezdy1/01_Vypadova_A41.csv',
 '../data/MultiDo/Eyedea/Prujezdy1/01_Vypadova_D600.csv',
 '../data/MultiDo/Eyedea/Prujezdy1/03_KPrehradam_A6000.csv',
 '../data/MultiDo/Eyedea/Prujezdy1/05_RX100.csv',
 '../data/MultiDo/Eyedea/Prujezdy1/06_RX100.csv',
 '../data/MultiDo/Eyedea/Prujezdy1/07_RX100.csv',
 '../data/MultiDo/Eyedea/Prujezdy1/08_iphone.csv',
 '../data/MultiDo/Eyedea/Prujezdy1/09_RX100.csv',
 '../data/MultiDo/Eyedea/Prujezdy1/10_RX100.csv',
 '../data/MultiDo/Eyedea/Prujezdy1/11_RX100.csv',
 '../data/MultiDo/Eyedea/Prujezdy1/12_RX100.csv',
 '../data/MultiDo/Eyedea/Prujezdy1/13_RX100.csv',
 '../data/MultiDo/Eyedea/Prujezdy1/14_RX100.csv',
 '../data/MultiDo/Eyedea/Prujezdy1/15_RX100.csv',
 '../data/MultiDo/Eyedea/Prujezdy1/16_RX100.csv',
 '../data/MultiDo/Eyedea/Prujezdy1/17_RX100.csv']

In [59]:
# os.symlink(src, dst)

In [64]:
names = []
for file in files:
    name = file.split('/')[-1]
#     print(name[:-4])
    names.append(name[:-4])
    name = f'{name[:-4]}.MP4.csv'
#     destination = f'data/csv/{name}'
#     print(destination)
#     shutil.copy2(file, destination)

In [70]:
all_files

['../data/MultiDo/dataset/12_Mobius.MOV',
 '../data/MultiDo/dataset/11_Mobius.MOV',
 '../data/MultiDo/dataset/10_RX100.csv',
 '../data/MultiDo/dataset/09_RX100.MTS.avi.mp4',
 '../data/MultiDo/dataset/16_iphone.mp4',
 '../data/MultiDo/dataset/05_Mobius.mov',
 '../data/MultiDo/dataset/16_Mobius.MOV',
 '../data/MultiDo/dataset/13_RX100.csv',
 '../data/MultiDo/dataset/08_Mobius.MOV',
 '../data/MultiDo/dataset/09_Mobius.MOV',
 '../data/MultiDo/dataset/11_iphone.MOV',
 '../data/MultiDo/dataset/10_iphone.MP4',
 '../data/MultiDo/dataset/17_RX100.MP4',
 '../data/MultiDo/dataset/03_KPrehradam_Mobius.MOV',
 '../data/MultiDo/dataset/07_iphone.mp4',
 '../data/MultiDo/dataset/03_KPrehradam_A41.mp4',
 '../data/MultiDo/dataset/03_KPrehradam_D600.MOV',
 '../data/MultiDo/dataset/06_Mobius.mov',
 '../data/MultiDo/dataset/11_RX100.MP4',
 '../data/MultiDo/dataset/07_RX100.csv',
 '../data/MultiDo/dataset/03_KPrehradam_iPhoneXS.mp4',
 '../data/MultiDo/dataset/06_RX100.csv',
 '../data/MultiDo/dataset/05_RX100

In [69]:
all_files = glob('../data/MultiDo/dataset/*')
for n in sorted(names):
    print('-',n)
    for file in all_files:
        if n in file:
            print(file)


- 01_Vypadova_A41
../data/MultiDo/dataset/01_Vypadova_A41.mp4
../data/MultiDo/dataset/01_Vypadova_A41.csv
- 01_Vypadova_D600
../data/MultiDo/dataset/01_Vypadova_D600.MOV
../data/MultiDo/dataset/01_Vypadova_D600.csv
- 03_KPrehradam_A6000
../data/MultiDo/dataset/03_KPrehradam_A6000.MP4
../data/MultiDo/dataset/03_KPrehradam_A6000.csv
- 05_RX100
../data/MultiDo/dataset/05_RX100.mp4
../data/MultiDo/dataset/05_RX100.csv
- 06_RX100
../data/MultiDo/dataset/06_RX100.csv
../data/MultiDo/dataset/06_RX100.mts
../data/MultiDo/dataset/06_RX100.mp4
- 07_RX100
../data/MultiDo/dataset/07_RX100.csv
../data/MultiDo/dataset/07_RX100.mp4
- 08_iphone
../data/MultiDo/dataset/08_iphone.csv
../data/MultiDo/dataset/08_iphone.MOV.mp4
- 09_RX100
../data/MultiDo/dataset/09_RX100.MTS.avi.mp4
../data/MultiDo/dataset/09_RX100.csv
- 10_RX100
../data/MultiDo/dataset/10_RX100.csv
../data/MultiDo/dataset/10_RX100.MTS.avi.mp4
- 11_RX100
../data/MultiDo/dataset/11_RX100.MP4
../data/MultiDo/dataset/11_RX100.csv
- 12_RX100
.